# Approximate H2 molecule energy near ground state

In this example notebook, we'll walk through calculating the energy of a molecule given an estimate for the ground state using Azure Quantum. We will do that using a simple algorithm where we prepare the quantum register in a state near the ground state of the molecule using Jordan-Wigner encoding, and evaluate the energy of the molecule by applying the Hamiltonian operator to that state. To be able to run the algorithm on near-term hardware, we will calculate the energy per Hamiltonian term, and then calculate the total energy by adding the values of these terms.

In [1]:
# First, import qsharp and qsharp.azure to be able to compile and submit the quantum program.
import qsharp
import qsharp.azure
qsharp.reload()
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTermH2, ExpandedCoefficients_

Preparing Q# environment...


Reloading workspace.

### 1. Load H2 molecule data

The pre-generated Broombridge file included in this sample contains the details on the molecule's Hamiltonian. A Hamiltonian is an operator that calculates the energy of a molecule by acting on a qubit register that represents the molecule's quantum state by Jordan-Wigner encoding. If we prepare the molecule in the ground state, then we know that applying the Hamiltonian will calculate the ground state energy.

In [2]:
from qdk.chemistry.broombridge import load_and_encode

Adding package microsoft.quantum.chemistry.jupyter.

In [3]:
encoded_data_h2 = load_and_encode("../data/broombridge/hydrogen_0.2.yaml")

This contains a tuple of the number of qubits, the fermionic Hamiltonian term coefficients and the energy offset.

The index of each list in the second tuple is will be referred to as the "term type".

In [4]:
num_qubits, fermion_terms, _, energy_offset = encoded_data_h2

### 2. Resource estimation
Before running on hardware, let's first estimate the quantum resources needed for measuring two-electron terms ("term type" = 3)

In [5]:
GetHamiltonianTermH2.estimate_resources(
    nOp=12
)

{'CNOT': 0,
 'QubitClifford': 2,
 'R': 0,
 'Measure': 5,
 'T': 0,
 'Depth': 0,
 'Width': 4,
 'QubitCount': 4,
 'BorrowedWidth': 0}

We have 11 qubits on the IonQ system so this is feasible to run!

### 3. Running on Azure Quantum

Connect to Azure Quantum

In [6]:
rid = "" # Enter your workspace's resource ID here
location = "West US" # Enter your workspace's location here, e.g. "West US"

In [7]:
qsharp.azure.connect(
   resourceId=rid,
   location=location
)

Connected to Azure Quantum workspace TestGuen in location westus.


[{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 11},
 {'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}]

In [8]:
qsharp.azure.target("ionq.simulator")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}

Run a single term on the IonQ simulator:

In [9]:
n_op = 17
result = qsharp.azure.execute(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}")

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 17
   Job ID: 1db9ee2e-f4ca-43e1-bf27-7d4f87681e67
Waiting up to 30 seconds for Azure Quantum job to complete...
[7:15:42 AM] Current job status: Succeeded


In [10]:
result

{'0': 0.5, '1': 0.5}

### 4. Loop over all Hamiltonian terms

Each Hamiltonian term has a term type, measurement operator and coefficient. To simplify this sample, we will move the coefficients into a flat list. Each nonzero coefficient will render a separate job that we send to Azure Quantum. To calculate the energy for that term, we map result from the job onto a -1 to 1 axis and multiply the value by the corresponding coefficient.

In [11]:
coeffs = []
for term_type, terms in enumerate(fermion_terms):
    for (qubits, coeff) in terms:
        coeffs += ExpandedCoefficients_(coeff=coeff, termType=term_type)
coeffs

[0.17120128499999998,
 0.17120128499999998,
 -0.222796536,
 -0.222796536,
 0.1686232915,
 0.12054614575,
 0.16586802525,
 0.16586802525,
 0.12054614575,
 0.1743495025,
 0.0,
 0.0,
 -0.0453218795,
 -0.0453218795,
 0.0,
 0.0,
 0.0453218795,
 0.0453218795]

There are in total 18 coefficients corresponding to each combination of Hamiltonian term and Jordan-Wigner measurement operator. Start a job for each term that has a nonzero coefficient:

In [18]:
jobs = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0.0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: ff06a236-700f-4293-be37-0597ff54f470
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: cf3a5c46-b33e-4fb2-8145-c33836051a16
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: 97b64b6b-0418-4651-9eaa-d259af443992
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: 31a8f734-2e58-4879-b9f7-0f53d98003ee
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job succ

In [19]:
results = [qsharp.azure.output(j.id) for j in jobs]
results

[{'1': 1.0},
 {'1': 1.0},
 {'0': 1.0},
 {'0': 1.0},
 {'0': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'0': 1.0},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5}]

### 5. Estimate energy

In [20]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)]) + energy_offset

In [21]:
calc_energy(coeffs, results)

-1.1166856359999997

Now we are ready to run on hardware! Switch to the QPU.

In [48]:
qsharp.azure.target("ionq.qpu")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 12252}

In [49]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: 7c8839d0-a553-44db-88a1-1c4591d19196
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: d73bcfc3-aefc-4a9f-8fc5-e71b60a74d30
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: bfd41d0e-96ca-4583-9aee-6425e76eddf8
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: e6f663c6-f23f-4db1-9973-ab83f6fdeaf0
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 sh

Check if the jobs succeeded.

In [5]:
jobs_status = [qsharp.azure.status(j) for j in jobs_hw]
[
    j.status + " | " + j.name + " | " + j.target + " | " + j.creation_time + " | " + j.end_execution_time 
    for j in jobs_status
]

['Succeeded | Hamiltonian term 0 | ionq.qpu | 2021-05-06T01:20:30.9254783Z | 2021-05-06T01:27:47.806Z',
 'Succeeded | Hamiltonian term 1 | ionq.qpu | 2021-05-06T01:20:39.1679175Z | 2021-05-06T01:27:56.779Z',
 'Succeeded | Hamiltonian term 2 | ionq.qpu | 2021-05-06T01:20:47.9310132Z | 2021-05-06T01:28:05.876Z',
 'Succeeded | Hamiltonian term 3 | ionq.qpu | 2021-05-06T01:20:58.5528459Z | 2021-05-06T01:28:15.744Z',
 'Succeeded | Hamiltonian term 4 | ionq.qpu | 2021-05-06T01:21:07.3776298Z | 2021-05-06T01:28:25.005Z',
 'Succeeded | Hamiltonian term 5 | ionq.qpu | 2021-05-06T01:21:16.1710463Z | 2021-05-06T01:28:34.117Z',
 'Succeeded | Hamiltonian term 6 | ionq.qpu | 2021-05-06T01:21:27.272853Z | 2021-05-06T01:28:43.245Z',
 'Succeeded | Hamiltonian term 7 | ionq.qpu | 2021-05-06T01:21:38.2052809Z | 2021-05-06T01:28:52.426Z',
 'Succeeded | Hamiltonian term 8 | ionq.qpu | 2021-05-06T01:21:49.4659481Z | 2021-05-06T01:29:01.519Z',
 'Succeeded | Hamiltonian term 9 | ionq.qpu | 2021-05-06T01:22:03

Get results and estimate energy

In [99]:
results = [qsharp.azure.output(j.id) for j in jobs_hw]
results

[{'0': 0.019, '1': 0.981},
 {'0': 0.016, '1': 0.984},
 {'0': 0.9770000000000001, '1': 0.023},
 {'0': 0.99, '1': 0.01},
 {'0': 0.9470000000000001, '1': 0.053},
 {'0': 0.057, '1': 0.9430000000000001},
 {'0': 0.030000000000000002, '1': 0.9700000000000001},
 {'0': 0.07300000000000001, '1': 0.927},
 {'0': 0.022, '1': 0.9780000000000001},
 {'0': 0.9780000000000001, '1': 0.022},
 {'0': 0.505, '1': 0.495},
 {'0': 0.511, '1': 0.48900000000000005},
 {'0': 0.491, '1': 0.509},
 {'0': 0.5, '1': 0.5}]

In [100]:
calc_energy(coeffs, results)

-0.9626770776219998